In [1]:
from huggingface_hub import notebook_login

notebook_login()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
import pandas as pd
import json
from pathlib import Path
from datasets import Dataset, DatasetDict


In [3]:
def load_splits(base_path: str, dataset_name: str):
    splits = ['train', 'valid', 'test']
    datasets = {}

    for split in splits:
        file_path = f"{base_path}/{dataset_name}-split_{split}.pkl"
        datasets[split] = pd.read_pickle(file_path)

    return datasets

In [4]:
mistral_datasets_configs = [
    {
        "mistral_dataset_path": "../outputs/dataset_pipeline/final_fq18_without_limit",
        "mistral_dataset_name": "final_fq18_without_limit",
        "hf_name": "labelized_sparql_wikidata_llama3_70B_no_limit"
    },
    {
        "mistral_dataset_path": "../outputs/dataset_pipeline/final_fq18_with_limit_10",
        "mistral_dataset_name": "final_fq18_with_limit_10",
        "hf_name": "labelized_sparql_wikidata_llama3_70B_limit_10"
    }
]

In [5]:
df_llama = pd.read_json("../outputs/dataset_pipeline/fq18_llama3/templated_generated_prompt_query_templated.json")
# df_llama.head()

In [6]:
for config in mistral_datasets_configs:
    splits = load_splits(config["mistral_dataset_path"], config["mistral_dataset_name"])
    
    for split, df_mistral in splits.items():
        df_mistral.loc[df_mistral.index.isin(df_llama.index), 'basic_input'] = df_llama['basic_result']
        df_mistral.loc[df_mistral.index.isin(df_llama.index), 'templated_input'] = df_llama['templated_result']
        
    hf_dataset = DatasetDict({k: Dataset.from_pandas(v, preserve_index=True) for k, v in splits.items()})
    hf_dataset.push_to_hub(config["hf_name"])


Uploading the dataset shards:   0%|          | 0/6 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
# new_splits = load_splits(config["mistral_dataset_path"], config["mistral_dataset_name"])

In [8]:
# mod_sample = splits["train"].sample()
# ori_sample = new_splits["train"].loc[mod_sample.index]
# print(f"mod_sample:\n{mod_sample['basic_input'].item()[0]}\n{mod_sample['templated_input'].item()[0]}")
# print(f"ori_sample:\n{ori_sample['basic_input'].item()[0]}\n{ori_sample['templated_input'].item()[0]}")